<a href="https://colab.research.google.com/github/tiutiutiu666/SC_Team-Assignment-Chapter-2-Machine-Learning-and-Deep-Learning/blob/main/DEEP%20LEARNING%20%5BCase%20%234%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment Chapter 2 - DEEP LEARNING [Case #4]
Startup Campus, Indonesia - `Artificial Intelligence (AI)` (Batch 7)
* Task: **CLASSIFICATION**
* DL Framework: **PyTorch**
* Dataset: Credit Card Fraud 2023
* Libraries: Pandas/cuDF, Scikit-learn/cuML, Numpy/cuPy
* Objective: Classify credit fraud transactions using Multilayer Perceptron

`PERSYARATAN` Semua modul (termasuk versi yang sesuai) sudah di-install dengan benar.
<br>`CARA PENGERJAAN` Lengkapi baris kode yang ditandai dengan **#TODO**.
<br>`TARGET PORTFOLIO` Peserta mampu mengklasifikasi transaksi fraud menggunakan *Multilayer Perceptron*

### Import Libraries

In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

fatal: destination path 'rapidsai-csp-utils' already exists and is not an empty directory.
Installing RAPIDS remaining 24.6.* libraries
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com

        ***********************************************************************
        The pip install of RAPIDS is complete.
        
        Please do not run any further installation from the conda based installation methods, as they may cause issues!
        
        Please ensure that you're pulling from the git repo to remain updated with the latest working install scripts.

        Troubleshooting:
            - If there is an installation failure, please check back on RAPIDSAI owned templates/notebooks to see how to update your personal files. 
            - If an installation failure persists when using the latest script, please make an issue on https://github.com/rapidsai-community/rapidsai-csp-utils
        *****************************************************************

In [ ]:
import shutil
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

<font color="red">**- - - - MOHON DIPERHATIKAN - - - -**</font>
<br>**Aktifkan GPU sekarang.** Di Google Colab, klik **Runtime > Change Runtime Type**, lalu pilih **T4 GPU**.

### Dataset Loading (CPU vs. GPU)

In [ ]:
from pandas import read_csv as read_by_CPU
from cudf import read_csv as read_by_GPU

In [ ]:
# unzip the file
shutil.unpack_archive('/content/dataset_case_04.zip')

In [ ]:
# TODO: Impor dataset dengan Pandas, gunakan fungsi "read_by_CPU"
%time data_cpu = read_by_CPU('/content/creditcard_2023.csv')

CPU times: user 3.43 s, sys: 323 ms, total: 3.76 s
Wall time: 3.76 s


In [ ]:
# Impor dataset dengan cuDF (Pandas di GPU)
%time data_gpu = read_by_GPU('/content/creditcard_2023.csv')

CPU times: user 325 ms, sys: 358 ms, total: 682 ms
Wall time: 890 ms


In [ ]:
data_gpu.head()

,id,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-0.260648,-0.469648,2.496266,-0.083724,0.129681,0.732898,0.519014,-0.130006,0.727159,...,-0.110552,0.217606,-0.134794,0.165959,0.126280,-0.434824,-0.081230,-0.151045,17982.10,0
1,1,0.985100,-0.356045,0.558056,-0.429654,0.277140,0.428605,0.406466,-0.133118,0.347452,...,-0.194936,-0.605761,0.079469,-0.577395,0.190090,0.296503,-0.248052,-0.064512,6531.37,0
2,2,-0.260272,-0.949385,1.728538,-0.457986,0.074062,1.419481,0.743511,-0.095576,-0.261297,...,-0.005020,0.702906,0.945045,-1.154666,-0.605564,-0.312895,-0.300258,-0.244718,2513.54,0
3,3,-0.152152,-0.508959,1.746840,-1.090178,0.249486,1.143312,0.518269,-0.065130,-0.205698,...,-0.146927,-0.038212,-0.214048,-1.893131,1.003963,-0.515950,-0.165316,0.048424,5384.44,0
4,4,-0.206820,-0.165280,1.527053,-0.448293,0.106125,0.530549,0.658849,-0.212660,1.049921,...,-0.106984,0.729727,-0.161666,0.312561,-0.414116,1.071126,0.023712,0.419117,14278.97,0


In [ ]:
# TODO: Hilangkan kolom ID
data_gpu = data_gpu.drop('id', axis = 1)

In [ ]:
data_gpu.sum()

V1       -1.455192e-11
V2       -3.637979e-11
V3        5.093170e-11
V4        0.000000e+00
V5        2.910383e-11
V6        1.091394e-11
V7        2.182787e-11
V8       -5.456968e-12
V9        1.455192e-11
V10       8.731149e-11
V11       7.275958e-12
V12      -1.164153e-10
V13       9.094947e-13
V14       1.455192e-11
V15       5.456968e-12
V16       2.182787e-11
V17       1.455192e-11
V18       1.455192e-11
V19      -1.091394e-11
V20      -2.910383e-11
V21       1.273293e-11
V22      -2.387424e-12
V23      -2.387424e-12
V24      -3.637979e-12
V25      -1.364242e-11
V26       1.818989e-12
V27      -2.182787e-11
V28       1.455192e-11
Amount    6.847418e+09
Class     2.843150e+05
dtype: float64

### Standardization (CPU vs. GPU)

In [ ]:
from sklearn.preprocessing import StandardScaler as StandardScalerCPU
from cuml.preprocessing import StandardScaler as StandardScalerGPU

In [ ]:
ScalerCPU = StandardScalerCPU()
ScalerGPU = StandardScalerGPU()

arbitrary_features = ["V"+str(i+1) for i in range(27)]

In [ ]:
%%time

data_cpu[arbitrary_features] = ScalerCPU.fit_transform(data_cpu[arbitrary_features].values)
data_cpu["Amount"] = ScalerCPU.fit_transform(data_cpu["Amount"].values.reshape(-1, 1)).squeeze()

CPU times: user 222 ms, sys: 422 ms, total: 644 ms
Wall time: 636 ms


In [ ]:
%%time

data_gpu[arbitrary_features] = ScalerGPU.fit_transform(data_gpu[arbitrary_features].values)
data_gpu["Amount"] = ScalerGPU.fit_transform(data_gpu["Amount"].values.reshape(-1, 1)).squeeze()

CPU times: user 16.9 s, sys: 1.24 s, total: 18.1 s
Wall time: 18.1 s


### Train/Test Split (CPU vs. GPU)

In [ ]:
from sklearn.model_selection import train_test_split as splitCPU
from cuml.preprocessing import train_test_split as splitGPU

In [ ]:
data_gpu.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-0.260648,-0.469648,2.496266,-0.083724,0.129681,0.732898,0.519014,-0.130006,0.727159,0.637735,...,-0.110552,0.217606,-0.134794,0.165959,0.126280,-0.434824,-0.081230,-0.151045,0.858447,0
1,0.985100,-0.356045,0.558056,-0.429654,0.277140,0.428605,0.406466,-0.133118,0.347452,0.529808,...,-0.194936,-0.605761,0.079469,-0.577395,0.190090,0.296503,-0.248052,-0.064512,-0.796369,0
2,-0.260272,-0.949385,1.728538,-0.457986,0.074062,1.419481,0.743511,-0.095576,-0.261297,0.690708,...,-0.005020,0.702906,0.945045,-1.154666,-0.605564,-0.312895,-0.300258,-0.244718,-1.377011,0
3,-0.152152,-0.508959,1.746840,-1.090178,0.249486,1.143312,0.518269,-0.065130,-0.205698,0.575231,...,-0.146927,-0.038212,-0.214048,-1.893131,1.003963,-0.515950,-0.165316,0.048424,-0.962119,0
4,-0.206820,-0.165280,1.527053,-0.448293,0.106125,0.530549,0.658849,-0.212660,1.049921,0.968046,...,-0.106984,0.729727,-0.161666,0.312561,-0.414116,1.071126,0.023712,0.419117,0.323285,0


In [ ]:
# TODO: Tentukan X (features) dan Y (target), gunakan "data_gpu"
X = data_gpu.drop('Class',1)
Y = data_gpu['Class']

In [ ]:
%%time

# TODO: Pecah dataset dengan komposisi 80% train set dan 20% test set, dengan fungsi "splitCPU"
test_size = 0.2
random_state = 42
x_train, x_test, y_train, y_test =  splitCPU(X, Y, test_size=test_size, random_state=random_state)

print("x_train shape: ", x_train.shape)
print("x_test shape: ", x_test.shape)

x_train shape:  (454904, 29)
x_test shape:  (113726, 29)
CPU times: user 63.7 ms, sys: 15.3 ms, total: 79.1 ms
Wall time: 134 ms


In [ ]:
%%time

# TODO: Lakukan hal yang sama untuk data spliting, tetapi dengan fungsi "splitGPU"
test_size = 0.2
random_state = 42
x_train, x_test, y_train, y_test = splitGPU(X, Y, test_size=test_size, random_state=random_state)

print("x_train shape: ", x_train.shape)
print("x_test shape: ", x_test.shape)

x_train shape:  (454904, 29)
x_test shape:  (113726, 29)
CPU times: user 1.43 s, sys: 91.8 ms, total: 1.52 s
Wall time: 1.59 s


### Convert the dataset into Tensor

In [ ]:
import cupy # Numpy for GPU

In [ ]:
# TODO: Aktifkan GPU (CUDA) sebagai device untuk training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
# Mengonversi cuDF ke CuPy, lalu ke NumPy, lalu ke PyTorch tensor dan pindahkan ke GPU
x_train_tensor = torch.from_numpy(cupy.asnumpy(x_train.to_cupy())).to(device)
y_train_tensor = torch.from_numpy(cupy.asnumpy(y_train.to_cupy())).to(device)

x_test_tensor = torch.from_numpy(cupy.asnumpy(x_test.to_cupy())).to(device)
y_test_tensor = torch.from_numpy(cupy.asnumpy(y_test.to_cupy())).to(device)

Train_tensor = TensorDataset(x_train_tensor, y_train_tensor)
Test_tensor = TensorDataset(x_test_tensor, y_test_tensor)

### Batching the Dataset with PyTorch DataLoader

In [ ]:
# TODO: Tentukan nilai batch
batch_size = 50

Train_dataset = DataLoader(Train_tensor, batch_size=batch_size, shuffle=True)
Test_dataset = DataLoader(Test_tensor, batch_size=batch_size, shuffle=False)

### Model Blueprint

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, input_dim, num_neurons):
        super(FeedForward, self).__init__()
        self.input_dim = input_dim
        self.num_neurons = num_neurons

        self.net = nn.Sequential(
            nn.Linear(self.input_dim, self.num_neurons),
            nn.ReLU()
        )

    def forward(self, x):
        return self.net(x)

    def to(self, device):
        self.net.to(device)
        return self

class Net(nn.Module):
    def __init__(self, in_features, num_layers, num_neurons):
        super(Net, self).__init__()
        self.in_features = in_features
        self.num_layers = num_layers
        self.num_neurons = num_neurons

        self.fc1 = nn.Linear(self.in_features, self.num_neurons)
        self.relu = nn.ReLU()
        self.blocks = [FeedForward(self.num_neurons, self.num_neurons).to(device) \
                       for _ in range(self.num_layers)]
        self.output_layer = nn.Linear(self.num_neurons, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        output = self.relu(self.fc1(x))

        for block in self.blocks:
            output = block(output)
        output = self.sigmoid(self.output_layer(output))

        return output

### Model Hyperparameters and Parameters

#### Pertanyaan: Apa perbedaan hyperparameters dan parameters?

Hyperparameters adalah sebuah variabel yang dapat kita tunning untuk memaksimalkan model sesuai yang kita butuhkan. Sementara parameter adalah sebuah variabel yang akan diupdate oleh sistem secara berkala selama menjalani training

Jika kita menggunakan analogi sederhana mungkin seperti :
- **Parameter** adalah bahan-bahan dalam resep, seperti tepung, gula, telur, dan
  mentega. Saat kamu mencampur bahan-bahan ini dan memanggang kue,
  bahan-bahan tersebut berubah (misalnya, tepung dan telur menjadi adonan). Ini mirip dengan bagaimana parameter dalam model diubah selama proses training untuk menghasilkan model yang baik.

- **Hyperparameter** adalah peralatan dan pengaturan dapur yang dipilih sebelum
  mulai memasak, seperti suhu oven, lama memanggang, atau ukuran loyang.
  Kamu yang mengatur hal-hal ini sebelum memulai, dan peralatan ini tidak berubah selama kamu memanggang kue. Sama seperti hyperparameter, yang diatur sebelum proses training dimulai, dan mempengaruhi hasil akhirnya.

Singkatnya, **hyperparameter** adalah pengaturan yang kita kendalikan, sedangkan **parameter** adalah apa yang model pelajari dari data.

Jadi, perbedaan utama antara keduanya adalah:
- **Hyperparameter** ditentukan sebelum pelatihan, sedangkan **parameter** dipelajari selama pelatihan.
- **Hyperparameter** mempengaruhi struktur dan proses pelatihan model, sedangkan **parameter** menentukan keluaran model.

In [ ]:
# TODO: Tentukan hyperparameters
epochs = 50
num_layers = 3
num_neurons = 64
learning_rate = 0.001

In [ ]:
# TODO: Tentukan besaran input untuk model
num_inputs = 29

model = Net(in_features=num_inputs, num_layers=num_layers, num_neurons=num_neurons)
model = model.to(device)

In [ ]:
# Set the optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.BCELoss()

In [ ]:
# Check the number of parameters
print("Number of parameters: {:,}".format(sum(p.numel() for p in model.parameters() if p.requires_grad)))
print("Number of trainable parameters: {:,}".format(sum(p.numel() for p in model.parameters() if p.requires_grad)))

Number of parameters: 1,985
Number of trainable parameters: 1,985


#### Pertanyaan: Mengapa total "trainable parameters" sama dengan total keseluruhan parameter?

Total trainable parameters sama dengan total keseluruhan parameters karena trainable parameters
adalah semua parameter dalam model yang dapat diubah atau dipelajari selama proses training



### Train the Model

In [ ]:
print("Start training ...")
for epoch in range(epochs):
    train_loss = 0.0
    model.train()

    for data, label in Train_dataset:
        data = data.to(device)
        label = label.squeeze()
        label = label.to(device)
        optimizer.zero_grad()
        output = model.forward(data.float())

        loss = criterion(output.squeeze(), label.float())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss = train_loss / len(Train_dataset.dataset)
    if(epoch % 10 == 0):
        print('  - Epoch: {} \tTraining_loss: {:.6f}'.format(epoch, train_loss))

Start training ...
  - Epoch: 0 	Training_loss: 0.001741
  - Epoch: 10 	Training_loss: 0.000038
  - Epoch: 20 	Training_loss: 0.000016
  - Epoch: 30 	Training_loss: 0.000010
  - Epoch: 40 	Training_loss: 0.000008


### Model ACCURACY should reach >= 95%

In [ ]:
# TODO: Jika akurasi masih dibawah 95%, silakan lakukan fine-tuning
print("""tidak, akurasi sudah lebih dari 95% jadi sudah cukup baik""")

tidak, akurasi sudah lebih dari 95% jadi sudah cukup baik


In [ ]:
correct_preds = 0
total_samples = 0

with torch.no_grad():
    for data, labels in Test_dataset:
        labels = labels.squeeze()
        output = model.forward(data.float())
        output = output.squeeze(1)

        predictions = (output >= 0.5).float()
        correct_preds += (predictions == labels).sum().item()
        total_samples += labels.numel()

accuracy = correct_preds / total_samples
print("Model accuracy: {:.2f}%".format(accuracy*100))

Model accuracy: 99.98%


In [ ]:
if accuracy >= 95 / 100:
  print(f"Akurasi sudah cukup baik, lihatlah : {accuracy * 100:.2f}%")
else:
  print(f"Akurasi belum cukup baik, silakan lakukan fine-tuning. Akurasi saat ini: {accuracy * 100:.2f}%")

Akurasi sudah cukup baik, lihatlah : 99.98%


### Scoring
Total `#TODO` = 12
<br>Checklist:

- [ ] Impor dataset dengan Pandas, gunakan fungsi "read_by_CPU"
- [ ] Hilangkan kolom ID
- [ ] Tentukan X (features) dan Y (target), gunakan "data_gpu"
- [ ] Pecah dataset dengan komposisi 80% train set dan 20% test set, dengan fungsi "splitCPU"
- [ ] Lakukan hal yang sama untuk data spliting, tetapi dengan fungsi "splitGPU"
- [ ] Aktifkan GPU (CUDA) sebagai device untuk training
- [ ] Tentukan nilai batch
- [ ] PERTANYAAN: Apa perbedaan hyperparameters dan parameters?
- [ ] Tentukan hyperparameters
- [ ] Tentukan besaran input untuk model
- [ ] PERTANYAAN: Mengapa total "trainable parameters" sama dengan total keseluruhan parameter?
- [ ] Jika akurasi masih dibawah 95%, silakan lakukan fine-tuning

### Additional readings
- N/A

### Copyright © 2024 Startup Campus, Indonesia
* Prepared by **Nicholas Dominic, M.Kom.** [(profile)](https://linkedin.com/in/nicholas-dominic)
* You may **NOT** use this file except there is written permission from PT. Kampus Merdeka Belajar (Startup Campus).
* Please address your questions to mentors.